Before running the file Upload all your data set on your goole drive in a zip format

In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


1
NVIDIA GeForce GTX 1050


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
url = '1ubvKLzBDe5i1acxgGUK6ObeNBYCKUS07'
gdd.download_file_from_google_drive(file_id = url,dest_path='./data.zip',unzip=True)

In [ ]:
#To get the average frame count 
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('/content/drive/MyDrive/DeepFake/Dataset/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

In [3]:
! pip install opencv-python

In [4]:
# RUNNING LOCALLY

import os
import numpy as np
import cv2

# Define the path to the video files
video_folder = "D:\\DeepFake\\Dataset"

# Get a list of all .mp4 files in the folder
video_files = [os.path.join(video_folder, file) for file in os.listdir(video_folder) if file.endswith('.mp4')]

# Initialize an empty list to store frame counts
frame_count = []

# Iterate over each video file
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    # Check if the video has at least 150 frames
    if int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150:
        continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

# Print the frame counts and average frame count
print("Frames per video:", frame_count)
print("Total number of videos:", len(frame_count))
print("Average frames per video:", np.mean(frame_count))


KeyboardInterrupt: 

With Cuda

In [6]:
#with cuda

import os
import numpy as np
import cv2
import torch

# Define the path to the video files
video_folder = "D:\\DeepFake\\Dataset"

# Get a list of all .mp4 files in the folder
video_files = [os.path.join(video_folder, file) for file in os.listdir(video_folder) if file.endswith('.mp4')]

# Initialize an empty list to store frame counts
frame_count = []

# Move operations to CUDA GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Iterate over each video file
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    # Check if the video has at least 150 frames
    if int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150:
        continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

# Print the frame counts and average frame count
print("Frames per video:", frame_count)
print("Total number of videos:", len(frame_count))
print("Average frames per video:", np.mean(frame_count))


Frames per video: [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 300, 301, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 299, 300, 300, 299, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 3

In [ ]:
import os
import numpy as np
import cv2
import torch
import face_recognition
from tqdm import tqdm

# Define the function to extract frames
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = 1
    while success:
        success, image = vidObj.read()
        if success:
            yield image

# Check if face_recognition is installed, if not, install it
try:
    import face_recognition
except ImportError:
    !pip install face_recognition
    import face_recognition

# Create directory to store processed videos
output_dir = 'D:/DeepFake/OnlyFace'
os.makedirs(output_dir, exist_ok=True)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Process the frames
def create_face_videos(path_list, out_dir):
    already_present_count = len(os.listdir(out_dir))
    print("Number of videos already present:", already_present_count)
    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.basename(path))
        if os.path.exists(out_path):
            print("File already exists:", out_path)
            continue
        frames = []
        flag = 0
        face_all = []
        frames1 = []
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (112, 112))
        for idx, frame in enumerate(frame_extract(path)):
            if idx <= 150:
                frames.append(frame)
                if len(frames) == 4:
                    faces = face_recognition.batch_face_locations(frames)
                    for i, face in enumerate(faces):
                        if len(face) != 0:
                            top, right, bottom, left = face[0]
                            try:
                                out.write(cv2.resize(frames[i][top:bottom, left:right, :], (112, 112)))
                            except:
                                pass
                    frames = []
        out.release()

# Call the function to process videos
create_face_videos(video_files, output_dir)


In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path) 
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/FF_REAL_Face_only_data'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/FF_REAL_Face_only_data/')